In [1]:
import tweepy
import configparser 
import pandas as pd
from tinydb import TinyDB, Query, Storage
import networkx as nx
import matplotlib as mpl
import matplotlib.pyplot as plt
import scipy
from pyvis.network import Network


In [2]:
config = configparser.ConfigParser(interpolation=None)
config.read('config.ini')

bearer_token = config['twitter']['bearer_token']
client = tweepy.Client(bearer_token=bearer_token, return_type = dict, wait_on_rate_limit=True)


### Getting a basic list of tweets

In [16]:
tweets = client.search_recent_tweets("barcelona", expansions=['author_id'], user_fields=['username'], max_results=10)

In [17]:
users_df = pd.DataFrame(tweets['includes']['users'])
data_df = pd.DataFrame(tweets['data'])
data_df = pd.merge(data_df, users_df, left_on=['author_id'], right_on=['id'])
data_df

,author_id,id_x,text,edit_history_tweet_ids,id_y,name,username
0,1071094302737088513,1604222981399609349,"RT @JordiGraupera: Ahir, després d'anar a veur...",[1604222981399609349],1071094302737088513,Cornelius☠,Corneli66092024
1,234015473,1604222978023198721,"RT @JordiGraupera: Ahir, després d'anar a veur...",[1604222978023198721],234015473,joanescapa - Més que escaldat,joanescapa
2,259396697,1604222969869467650,"RT @BoigBCN: Portal de l'Àngel. Barcelona, Nad...",[1604222969869467650],259396697,Toni Vall,tonivall
3,619181925,1604222932502433792,RT @tallerlutier: La presència dels instrument...,[1604222932502433792],619181925,Josep Maria Rebés,JMRebes
4,3296831620,1604222921739849728,RT @Tomiconcina1: Maradona se fue del Barcelon...,[1604222921739849728],3296831620,Tona,TonarelliAgus
5,1081520083,1604222921517301760,Acabo de comprar el pasaje de mi hermanita par...,[1604222921517301760],1081520083,Clau,claubabylongirl
6,1283207016585068547,1604222918749200385,Llegaron al límite de fracturarlo en un partid...,[1604222918749200385],1283207016585068547,Matemático hípico,Matematicohipi1
7,2150299375,1604222906975895554,RT @LaRepublicaCat: L'alcaldable d'ERC a la ci...,[1604222906975895554],2150299375,ramon bm,ramonbamo
8,1458046672580358147,1604222905709215744,RT @SocialDrive_es: Roban a un conductor mient...,[1604222905709215744],1458046672580358147,aprendelotodoytiraloalabasura,AprendelotodoY
9,984840763715530752,1604222897530224642,Bona nit 😉🎄 #streetphotography #fotografia #Ba...,[1604222897530224642],984840763715530752,La Chumbera,LChumbera


### Analyzing users declared networks

#### Getting all followers and following relationships for the users

We first obtain a small dataframe with three users. The first from the previous dataframe, a follower, and a follower from this second user.

In [20]:
graph_users_df = pd.DataFrame()

graph_users_df1 = pd.DataFrame(client.get_users_followers(users_df.iloc[0]['id'])['data'][0], index=[0])
graph_users_df2 = pd.DataFrame(client.get_users_followers(graph_users_df1.iloc[0]['id'])['data'][0], index=[0])
graph_users_df3 = pd.DataFrame(client.get_users_followers(graph_users_df2.iloc[0]['id'])['data'][0], index=[0])
graph_users_df = pd.concat([graph_users_df1, graph_users_df2, graph_users_df3])
graph_users_df

Rate limit exceeded. Sleeping for 792 seconds.


Then we get all of these users in a simple data structure represented on a document oriented nosql database (Tinydb). The structure looks like:

"3391": {"id": "1508297808000032776", "username": "KeavieRominger", "name": "Keavie Rominger", "followers": [ " list of followers "], "followed_by": [ "list of accounts the user is following" ] }

In [6]:
!rm ./db.json
db = TinyDB('./db.json')

for index, row in graph_users_df.iterrows():

    followers_list = []
    for response in tweepy.Paginator(client.get_users_followers,
                                    id = row['id'],
                                    max_results=1000, 
                                    limit=10):
        for user in response['data']:
            followers_list.append(user)

    followed_by_list = []
    for response in tweepy.Paginator(client.get_users_following,
                                    id = row['id'],
                                    max_results=1000, 
                                    limit=10):
        for user in response['data']:
            followed_by_list.append(user)

    User = Query()

    for follower in followers_list:
        if not (db.search(User.username == follower['username'])):
            db.insert({'id': follower['id'], 'username': follower['username'], 'name': follower['name']})

    for followed in followed_by_list:
        if not (db.search(User.username == followed['username'])):
            db.insert({'id': followed['id'], 'username': followed['username'], 'name': followed['name']}) 

    if not (db.search(User.username == row['username'])):
        db.insert({'id': row['id'], 'username': row['username'], 'name': row['name'], 'followers': 
            [d['username'] for d in followers_list], 'followed_by': [d['username'] for d in followed_by_list]})
    else:
        db.update_multiple([
            ({'followers': [d['username'] for d in followers_list]}, User.username == row['username']),
            ({'followed_by': [d['username'] for d in followed_by_list]}, User.username == row['username']),
        ])

#### Graph generation

In [ ]:
G = nx.DiGraph()

User = Query()
users_w_followers = db.search(User.followers.exists())
users_w_followed_by = db.search(User.followed_by.exists())

for user in users_w_followers:
        for follower in user['followers']:
                G.add_edge(follower,  user['username'], color='green')

for user in users_w_followed_by:
        for followed_by in user['followed_by']:
                G.add_edge(user['username'], followed_by, color='blue')

nt = Network('1300px', '600px', select_menu = True, directed=True)
nt.from_nx(G)
nt.show_buttons(filter_=['physics'])
nt.show('nx.html')